### 0. Multiple APIs

TensorFlow provides multiple APIs.
1. Lowest level - TensorFlow Core 
Complete programming control.

2. Higher level APIs
- Built on top of TensorFlow Core. 
- Typically easier to learn and use than Core.
- Make repetitive tasks easier and more consistent btw dif users.


### 1. Tensors

The central unit of data in TensorFlow is the **tensor.**

A tensor consists of a set of primitive values shaped into an array of any number of dimensions.

A tensor's **rank** is its number of dimensions. here are some examples of tensors :

In [18]:
3 # a rank 0 tensor; a scalar with shape []

[1., 2., 3.] # a rank 1 tensor; a vector with shape [3]

[[1., 2., 3.], [4., 5., 6.]] # a rank 2 tensor; a matrix with shape [2, 3]

[[[1., 2., 3.]], [[7., 8., 9.]]] # a rank 3 tensor with shape [2, 1, 3]#

[[[1.0, 2.0, 3.0]], [[7.0, 8.0, 9.0]]]

In [19]:
# The canonical import statement for TensorFlow is as follows :

import tensorflow as tf

You might think of Core as consisting of two discrete sections :
1. Building the computational graph.


2. Running the computational graph.

### 2. Computational graph

is a series of TensorFlow operations arranged into a graph of nodes.


Let's build a simple computational graph.

In [20]:
# Each node takes zero or more tensors as inputs and produces a tensor as an output.

# One type of node is a constant
    # It takes no inputs
    # and it outputs a value it sotres internally

node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly
print(node1, node2)

Tensor("Const_4:0", shape=(), dtype=float32) Tensor("Const_5:0", shape=(), dtype=float32)


#### 1) Session
Notice that printing the nodes does not output the values 3.0 and 4.0 as you might expect.

Instead, they are nodes that, when evaluated, would produce 3.0 and 4.0. 

To actually evaluate the nodes, we must run the computational graph ***within a session.***

** A session encapsulates the control and state of the TensorFlow runtime. **


In [21]:
# The following code creates a session object 
# and then invokes its run method to run computational graph 
# to evaluate node1 and node 2

sess = tf.Session()
print(sess.run([node1, node2]))

[3.0, 4.0]


In [22]:
# We can build more complicated computations by combining Tensor nodes with operations.
# For example, we can add our two constant nodes and produce a new graph as follows :

from __future__ import print_function
node3 = tf.add(node1, node2)
print("node3:", node3)
print("sess.run(node3):", sess.run(node3))

node3: Tensor("Add_1:0", shape=(), dtype=float32)
sess.run(node3): 7.0


TensorFlow provides a utility called **TensorBoard** that can display a picture of the computational graph.

#### 2) Placeholder
As it stands, this graph is not especially interesting because it always produces a constant result. 

A graph can be parameterized to accept external inputs, known as **placeholders**.

A placeholder is a promise to provide a value later.

In [23]:
# following three lines are a bit llike a function or a lambda
# in which we define two input parameters and do operation on them
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a+b  # it's a shortcut for tf.add(a,b)

We can evaluate this graph with multiple inputs by using the feed_dict arg to the run method to feed concrete values to the placeholder.

In [24]:
print(sess.run(adder_node, {a: 3, b:4.5}))
print(sess.run(adder_node, {a: [1,3], b: [2,4]}))

7.5
[ 3.  7.]


In [25]:
# more complex by adding another operation
add_and_triple = adder_node * 3.
print(sess.run(add_and_triple, {a:3, b:4.5}))

22.5


#### 3) Variables
In machine learning, we will typically want a model that can take arbitrary inputs.

To make the model trainable, we need to be able to modify the graph to get new outputs with the same input.

**Variables** allow us to add trainable parameters to a graph.

In [26]:
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W*x + b
linear_model

<tf.Tensor 'add_3:0' shape=<unknown> dtype=float32>

Constants are initialized when you call tf.constant, and their value can never change. 

By contrast, variables are not initialized when you call tf.Variable. 

To initialize all the variables in a TensorFlow program, you must explicitly call a special operations as follows :

In [27]:
init = tf.global_variables_initializer()
sess.run(init)

It is important to realize init is a handle to the TensorFlow sub-graph that initializes all the global variables. 

Variables can be reassigned using **tf.assign**

Since 'x' is a placeholder, we can evaluate 'linear_model' for several values of 'x' as follows :

In [28]:
print(sess.run(linear_model, {x:[1,2,3,4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


We've created a model, but we don't know how good it is yet. 

To evaluate the model on training data, we need a 'y' placeholder to provide the desired values, and we need to write a loss function.

#### 4) Loss function

A loss function measures how far apart the current model is from the provided data.

We'll use a standard loss model for linear regression, which sums the squares of the deltas btw the current model and the provided data.

'linear_model - y ' creates a vector where each element is the corresponding example's error delta.

We call **tf.square** to square that error. Then, we sum all the sqrd errors to create a single scalar using **tf.reduce_sum**.

In [29]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
print(sess.run(loss, {x:[1, 2, 3, 4], y: [0, -1, -2, -3]}))

23.66


We could improve this manually by reassigning the values of W and b to the perfect values of -1 and 1.

In [32]:
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb])
print(sess.run(loss, {x: [1, 2, 3, 4], y : [0, -1, -2, -3]}))

#Now the loss is zero.

0.0


### 3. if.train API

Tensorflow provides **optimizers** that slowly change each variable in order to minimize the loss function.

The simplest optimizer is **gradient descent.** 

Let's look at some example :

In [35]:
# Gradient Descent
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

sess.run(init)  # reset values to incorrect defaults.
for i in range(1000) :
    sess.run(train, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]})

print(sess.run([W, b]))

# results is the final parameter values (W and b)

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


# 아래 부분 아직 안했음